# Imports

In [1]:
# Python packages
import sys
sys.path.append('../../')
from datetime import datetime
import numpy as np
import pandas as pd

# Custom functions
import src.settings as settings
import src.mapper_cols as mapper_cols
from src.run_all.main_get_data import get_data
from src.run_all.main_preprocess import preprocess_data
from src.utilities.utilities import get_latest_file, list_filenames

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Get data
This step will load and combine several tables from CBS statline. 

Note: This step takes a number of minutes and without changes to the settings will give the same result. Therefor this code is commented out and the original dataset is loaded. 

In [2]:
%%time 
# ## CREATE NEW DATASET
# df_get_data_WMO= get_data(save=True)

# ## HARDCODED
datapath = '../../data/'
filename = 'df_get_data_WMO_WIJK_HUISHOUDENS_BEVOLKING_HEFFING_202104042111.parquet.gzip'
df_get_data_WMO = pd.read_parquet(datapath + filename)

# ## SELECT LAST FILE
# datapath = '../../data/'
# df = get_latest_file(filename_str_contains='df_WMO_', datapath=datapath, filetype='parquet')

print(f"The shape of the dataframe from step 'Get Data': {df_get_data_WMO.shape}")
df_get_data_WMO.sample(5)

The shape of the dataframe from step 'Get Data': (936, 470)
CPU times: user 151 ms, sys: 37.8 ms, total: 189 ms
Wall time: 184 ms


,,perioden,typemaatwerkarrangement,wmoclienten,wmoclientenper1000inwoners,aantalinkomensontvangers,aantalinwoners,actieven1575jaar,afstandtotgrotesupermarkt,afstandtothuisartsenpraktijk,afstandtotkinderdagverblijf,afstandtotschool,alandbouwbosbouwenvisserij,bedrijfsmotorvoertuigen,bedrijfsvestigingentotaal,bevolkingsdichtheid,bfnijverheidenenergie,bouwjaarvanaf2000,bouwjaarvoor2000,dekkingspercentage,eenpersoonshuishoudens,eigendomonbekend,geboorterelatief,geboortetotaal,gehuwd,gemeentenaam,gemelectriciteitsverbruikappartement,gemelectriciteitsverbruikeigenwoning,gemelectriciteitsverbruikhoekwoning,gemelectriciteitsverbruikhuurwoning,gemelectriciteitsverbruiktussenwoning,gemelectriciteitsverbruiktweeondereenkapwoning,gemelectriciteitsverbruikvrijstaandewoning,gemgasverbruikappartement,gemgasverbruikeigenwoning,gemgasverbruikhoekwoning,gemgasverbruikhuurwoning,gemgasverbruiktussenwoning,gemgasverbruiktweeondereenkapwoning,gemgasverbruikvrijstaandewoning,gemgestandaardiseerdinkomenvanhuish,gemiddeldaardgasverbruiktotaal,gemiddeldehuishoudensgrootte,gemiddeldelektriciteitsverbruiktotaal,gemiddeldewoningwaarde,gemiddeldinkomenperinkomensontvanger,gemiddeldinkomenperinwoner,gescheiden,geweldsenseksuelemisdrijven,gihandelenhoreca,hjvervoerinformatieencommunicatie,huishonderofrondsociaalminimum,huishoudensmeteenlaaginkomen,huishoudensmetkinderen,huishoudenstot110vansociaalminimum,huishoudenstot120vansociaalminimum,huishoudenstotaal,huishoudenszonderkinderen,huurwoningentotaal,inbezitoverigeverhuurders,inbezitwoningcorporatie,jongerenmetjeugdzorginnatura,k0tot15jaar,k15tot25jaar,k20huishoudensmethoogsteinkomen,k20personenmethoogsteinkomen,k25tot45jaar,k40huishoudensmetlaagsteinkomen,k40personenmetlaagsteinkomen,k45tot65jaar,k65jaarofouder,klfinancieledienstenonroerendgoed,koopwoningen,mannen,marokko,matevanstedelijkheid,mediaanvermogenvanparticulierehuish,meestvoorkomendepostcode,mnzakelijkedienstverlening,motorfietsen,nederlandseantillenenaruba,nettoarbeidsparticipatie,nietwesterstotaal,omgevingsadressendichtheid,ongehuwd,opleidingsniveauhoog,opleidingsniveaulaag,opleidingsniveaumiddelbaar,oppervlakteland,oppervlaktetotaal,oppervlaktewater,overignietwesters,percentagebewoond,percentageeengezinswoning,percentagejongerenmetjeugdzorg,percentagemeergezinswoning,percentageonbewoond,percentagewerknemers,percentagewoningenmetstadsverwarming,percentagezelfstandigen,personenautos6jaarenouder,personenautosbrandstofbenzine,personenautosjongerdan6jaar,personenautosnaaroppervlakte,personenautosoverigebrandstof,personenautosperhuishouden,personenautostotaal,personenpersoortuitkeringao,personenpersoortuitkeringaow,personenpersoortuitkeringbijstand,personenpersoortuitkeringww,rucultuurrecreatieoverigediensten,scholenbinnen3km,sterfterelatief,sterftetotaal,suriname,totaaldiefstaluitwoningschuured,turkije,vernielingmisdrijftegenopenbareorde,verweduwd,vrouwen,westerstotaal,woningvoorraad,alleenstaande_mannen,alleenstaande_totaal_mannen_en_vrouwen,alleenstaande_vrouwen,ouder_in_eenouderhuishouden_mannen,ouder_in_eenouderhuishouden_totaal_mannen_en_vrouwen,ouder_in_eenouderhuishouden_vrouwen,overig_lid_huishouden_mannen,overig_lid_huishouden_totaal_mannen_en_vrouwen,overig_lid_huishouden_vrouwen,"partner_in_paar,_geen_kind(eren)_mannen","partner_in_paar,_geen_kind(eren)_totaal_mannen_en_vrouwen","partner_in_paar,_geen_kind(eren)_vrouwen",partner_in_paar_met_kind(eren)_mannen,partner_in_paar_met_kind(eren)_totaal_mannen_en_vrouwen,partner_in_paar_met_kind(eren)_vrouwen,persoon_in_institutioneel_huishouden_mannen,persoon_in_institutioneel_huishouden_totaal_mannen_en_vrouwen,persoon_in_institutioneel_huishouden_vrouwen,persoon_in_particulier_huishouden_mannen,persoon_in_particulier_huishouden_totaal_mannen_en_vrouwen,persoon_in_particulier_huishouden_vrouwen,thuiswonend_kind_mannen,thuiswonend_kind_totaal_mannen_en_vrouwen,thuiswonend_kind_vrouwen,totaal_personen_mannen,totaal_personen_totaal_mannen_en_vrouwen,totaal_personen_vrouwen,popaantalbasisonderwijs

# Preprocess
This step will transform (select columns, impute, scale) the dataframe to be used in train/predict. 

In the dictionary `testsets` the `key` will be used to identify the preprocessed data file and the `value` is a list, containting three elements:
* The first element is a `dictionary` to use in the `RelativeColumnScaler`. The `key` is the columnname of the base variable and the `value` a list of columnnames which can be divided by the base variable. 
* The second element is a `list` with the columnnames to be scaled by the `CustomScaler`.
* The last element is a `list` with the columnnames to keep (last step in the pipeline; `ColumnSelector`).

If there are more steps which a user wants to change, the settings of preprocess are visible by running the code `settings.preprocess`. By assigning a new value to a key in the dictionary, the preprocess pipeline will be changed.

Each pipeline will lead to a parquet file and a log file in the folder `data/log_preprocess`. 

In [8]:
testsets = {'All': [mapper_cols.DICT_WMO_RELATIVELY_COLS_ALL, 
                          mapper_cols.LIST_WMO_GET_DATA_ALL, 
                          None],
            'No_Relative': [{}, 
                          mapper_cols.LIST_WMO_GET_DATA_ALL, 
                          None],
            'Boerenverstand_Maikel': [mapper_cols.DICT_WMO_RELATIVELY_COLS_BOERENVERSTAND_MAIKEL, 
                                      mapper_cols.LIST_WMO_GET_DATA_BOERENVERSTAND_MAIKEL, 
                                      mapper_cols.LIST_COLUMNSELECTOR_2_BOERENVERSTAND_MAIKEL],
           'Minimum_Maikel': [mapper_cols.DICT_WMO_RELATIVELY_COLS_BOERENVERSTAND_MAIKEL, 
                              mapper_cols.LIST_WMO_GET_DATA_BOERENVERSTAND_MAIKEL,
                             ['codering_regio', 'interval', 'wmoclienten', 'wmoclientenper1000inwoners',
                              'aantalinwoners','gescheiden','verweduwd','alleenstaande_mannen',
                              'alleenstaande_vrouwen','poparbeidsongeschiktheidtotaal',
                              'popbevolkingsdichtheid','popk65tot80jaarrelatieveleeftijdsgroep',
                              'popk80jaarofouderrelatieveleeftijdsgroep']]}
            

In [9]:
for key, value in testsets.items():
    print(f"Testset {key} will be created")
    # Set the parameters
    settings.preprocess['DICT_RELATIVELY_COLS'] = value[0]
    settings.preprocess['LIST_CUSTOMSCALER_COLS'] = value[1]
    settings.preprocess['LIST_COLUMNSELECTOR_COLS_2'] = value[2]
    # Create set
    df_preprocessed = preprocess_data(df=df_get_data_WMO, save_all=True, personal_note=key)
    print(f"The shape of the dataframe from step 'Preprocess': {df_preprocessed.shape}")

Testset All will be created
The shape of the dataframe from step 'Preprocess': (936, 510)
Testset No_Relative will be created
The shape of the dataframe from step 'Preprocess': (936, 340)
Testset Boerenverstand_Maikel will be created
The shape of the dataframe from step 'Preprocess': (936, 43)
Testset Minimum_Maikel will be created
The shape of the dataframe from step 'Preprocess': (936, 11)


# Appendix
Some backup of code used previously and may come in handy later.


## Code examples to get a subset of the DataFrame based on multiindex

In [5]:
# # Subset on columnvalue:
# df[df['gemeentenaam']=='Nijmegen']

# # One row / record
# df.loc[('WK026801', '2019')]

# # Multiple rows / records based on combination of the multiindex
# df.loc[[('WK026801', '2018'), ('WK026802', '2018')]]

# # Multiple rows / records for one column (works only for series)
# df['codering_regio'].loc[(['WK026801', 'WK026802'], ['2018', '2019'])]

# # Multiple rows based on both indexes:
# df.loc(axis=0)[['WK026801', 'WK026802'], ['2018', '2019']]

# # Subset with IndexSlice
# idx = pd.IndexSlice
# # Subset on one of the multiindex and select a column
# df.loc[idx[:, ['2018', '2019']], idx["codering_regio"]]
# # Subset on one of the multiindex and select all columns
# df.loc[idx['WK026801', :], idx[:]]

In [6]:
# df[df['gemeentenaam']=='Nijmegen']

## Code to get current versions of loaded packages

In [ ]:
# print('\n'.join(f'{m.__name__} {m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))

## Create a huge dictionary with all possible columns to scale with RelativeColumnScaler

In [ ]:
# # Create a huge dictionary with all possible columns to scale with RelativeColumnScaler
# dict_combined = {}
# dicts = [mapper_cols.DICT_RELATIVELY_COL_WIJK,
#          mapper_cols.DICT_RELATIVELY_COL_HUISHOUDEN,
#         mapper_cols.DICT_RELATIVELY_COL_BEVOLKING,
#         mapper_cols.DICT_RELATIVELY_COL_HEFFING]

# for D in dicts:
#     for key, value in D.items():
#         if key in dict_combined.keys():
#             value_dict_combined = dict_combined[key]
#             new_value = list(set(value_dict_combined+value))
#             dict_combined[key] = new_value
#         else:
#             dict_combined[key] = value
# dict_combined

## Review logging

In [ ]:
# ## combineren van de verschillende best model properties csv's naar één dataframe
# all_filenames = list_filenames(settings.preprocess['LOG_PATH'], filename_str_contains='.csv')
# combined_logging = pd.concat([pd.read_csv(f"{settings.preprocess['LOG_PATH']}{f}") for f in all_filenames ])
# combined_logging